In [2]:
import pandas as pd

In [3]:
# now that we have the df for twitter sentiment of each day fron 20170101
sentiment_filename = 'dailysentiments.csv'
stock_filename = 'stockprice_raw.csv'
sentiments = pd.read_csv(sentiment_filename, engine='python',index_col='date')
stocks = pd.read_csv(stock_filename, engine = 'python',index_col='Date')
sentiments = sentiments.drop(columns=sentiments.columns[0])

In [4]:
# set the size of the sliding window
window_size = 2
# use Close_prev to find the close price two valid days ago
stocks['Close_prev'] = stocks['Close'].shift(2)

In [5]:
stocks

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Close_prev
Date,,,,,,,,,
2016-12-30,20161230,43.259998,43.500000,42.335999,42.737999,23213000,0,0.0,NaN
2017-01-03,20170103,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0,NaN
2017-01-04,20170104,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0,42.737999
2017-01-05,20170105,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0,43.397999
2017-01-06,20170106,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0,45.397999
...,...,...,...,...,...,...,...,...,...
2020-12-28,20201228,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0,645.979980
2020-12-29,20201229,661.000000,669.900024,655.000000,665.989990,22910800,0,0.0,661.770020
2020-12-30,20201230,672.000000,696.599976,668.359985,694.780029,42846000,0,0.0,663.690002


In [10]:
# need to work out the weekends' label

In [6]:
stocks['Diff'] = stocks['Open'] - stocks['Close_prev']
stocks.loc[stocks['Diff']> 0, 'Diff'] = 1
stocks.loc[stocks['Diff']< 0, 'Diff'] = -1
stocks.loc[stocks['Diff'] == 0, 'Diff'] = 0
stock_label_df = stocks[['date','Diff']]

In [7]:
stock_label_df
# stock_label_df.to_csv('stockprice_label.csv')

,date,Diff
Date,,
2016-12-30,20161230,NaN
2017-01-03,20170103,NaN
2017-01-04,20170104,1.0
2017-01-05,20170105,1.0
2017-01-06,20170106,-1.0
...,...,...
2020-12-28,20201228,1.0
2020-12-29,20201229,-1.0
2020-12-30,20201230,1.0


In [8]:
# Create a df with sentiment analysis from previous two days given the dateindex 
sentiment_prev2days = pd.concat([sentiments.shift(1),sentiments.shift(2)],axis=1)
sentiment_prev2days = sentiment_prev2days.dropna()
sentiment_prev2days.insert(0,'date', sentiment_prev2days.index)
sentiment_prev2days.index.names = ['Date']
sentiment_prev2days
# note that 1448 tweet days vs. 1007 valid stock labels

,date,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd
Date,,,,,,,,,,,,,
20170103,20170103,0.405631,0.297854,0.526066,0.270588,0.068304,0.100616,0.389289,0.278815,0.550918,0.271281,0.059792,0.132426
20170104,20170104,0.204310,0.305864,0.369249,0.198773,0.426441,0.289451,0.405631,0.297854,0.526066,0.270588,0.068304,0.100616
20170105,20170105,0.260648,0.209138,0.620133,0.230882,0.119219,0.241072,0.204310,0.305864,0.369249,0.198773,0.426441,0.289451
20170106,20170106,0.397876,0.232893,0.522812,0.168254,0.079312,0.167570,0.260648,0.209138,0.620133,0.230882,0.119219,0.241072
20170107,20170107,0.313490,0.229368,0.585565,0.234168,0.100946,0.230763,0.397876,0.232893,0.522812,0.168254,0.079312,0.167570
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201227,20201227,0.371328,0.354282,0.497004,0.325460,0.131668,0.243043,0.438897,0.367852,0.421036,0.305266,0.140067,0.232719
20201228,20201228,0.495777,0.365542,0.399551,0.304094,0.104671,0.213257,0.371328,0.354282,0.497004,0.325460,0.131668,0.243043
20201229,20201229,0.408934,0.347921,0.457482,0.300991,0.133583,0.246821,0.495777,0.365542,0.399551,0.304094,0.104671,0.213257


In [9]:
dataframe = sentiment_prev2days.merge(stock_label_df,how='left', left_on='date', right_on='date')
dataframe

,date,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd,pos_mean,pos_sd,neu_mean,neu_sd,neg_mean,neg_sd,Diff
0,20170103,0.405631,0.297854,0.526066,0.270588,0.068304,0.100616,0.389289,0.278815,0.550918,0.271281,0.059792,0.132426,NaN
1,20170104,0.204310,0.305864,0.369249,0.198773,0.426441,0.289451,0.405631,0.297854,0.526066,0.270588,0.068304,0.100616,1.0
2,20170105,0.260648,0.209138,0.620133,0.230882,0.119219,0.241072,0.204310,0.305864,0.369249,0.198773,0.426441,0.289451,1.0
3,20170106,0.397876,0.232893,0.522812,0.168254,0.079312,0.167570,0.260648,0.209138,0.620133,0.230882,0.119219,0.241072,-1.0
4,20170107,0.313490,0.229368,0.585565,0.234168,0.100946,0.230763,0.397876,0.232893,0.522812,0.168254,0.079312,0.167570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,20201227,0.371328,0.354282,0.497004,0.325460,0.131668,0.243043,0.438897,0.367852,0.421036,0.305266,0.140067,0.232719,NaN
1444,20201228,0.495777,0.365542,0.399551,0.304094,0.104671,0.213257,0.371328,0.354282,0.497004,0.325460,0.131668,0.243043,1.0
1445,20201229,0.408934,0.347921,0.457482,0.300991,0.133583,0.246821,0.495777,0.365542,0.399551,0.304094,0.104671,0.213257,-1.0
1446,20201230,0.429391,0.341881,0.426625,0.280735,0.143984,0.249932,0.408934,0.347921,0.457482,0.300991,0.133583,0.246821,1.0
